# **March Madness**

<img src="https://a.espncdn.com/photo/2025/0316/2025_ncaam_bracket_filled_3x2.jpg" />

In [3]:
import pandas as pd

## CBB Data

**Columns**
 - CONF: Conference
 - G: Games played
 - W: Number of wins
 - ADJOE: Adjusted offensive efficiency
 - ADJDE: Adjusted defensive efficiency
 - BARTHAG: Power rating
 - EFG%: Effective field goal percentage
 - EFGD%: Effective field goal percentage allowed
 - TOR: Turnover rate
 - TORD: Forced turnover rate
 - ORB: Offensive rebound rate
 - DRB: Defensive rebound rate
 - FTR: Free throw rate
 - FTRD: Free throw rate allowed
 - 2P_O: Two-point shooting percentage
 - 2P_D: Two-point shooting percentage allowed
 - 3P_O: Three-point shooting percentage
 - 3P_D: Three-point shooting percentage allowed
 - ADJ_T: Adjusted tempo
 - WAB: Wins above bubble
 - Postseason: NCAA tournament finish
 - Seed: Seed in NCAA tournament
 

In [4]:
cbb = pd.read_csv("/Users/jonathan/cmse401/Project/CMSE401-Project/data/cbb.csv")
cbb = cbb.dropna()
cbb

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3223,Northern Kentucky,Horz,33,22,102.6,101.4,0.5349,49.1,51.3,17.3,...,30.5,47.4,49.3,34.3,36.4,61.9,-6.7,R64,16.0,2023
3224,Howard,MEAC,32,22,102.7,106.3,0.4031,51.8,50.9,22.8,...,33.9,49.8,51.1,36.9,33.8,69.0,-7.3,R64,16.0,2023
3225,Texas A&M Corpus Chris,Slnd,31,24,105.8,107.4,0.4566,50.5,52.6,17.7,...,38.2,48.3,54.1,36.4,33.8,68.9,-6.0,R64,16.0,2023
3226,Texas Southern,SWAC,33,14,95.4,105.4,0.2394,46.1,49.6,20.3,...,38.8,47.7,48.9,28.0,33.9,69.2,-13.5,R68,16.0,2023


In [5]:
cbb24 = pd.read_csv("/Users/jonathan/cmse401/Project/CMSE401-Project/data/cbb24.csv")
cbb24 = cbb24.dropna()
cbb24

,RK,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG%,EFGD%,...,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED
0,1,Houston,B12,34,30,119.2,85.5,0.9785,49.7,44.0,...,29.9,39.0,48.4,43.4,34.7,30.0,63.3,10.6,S16,1.0
1,2,Connecticut,BE,34,31,127.1,93.6,0.9712,57.1,45.1,...,33.3,32.5,58.5,43.7,36.7,31.9,64.6,11.3,Champions,1.0
2,3,Purdue,B10,33,29,126.2,94.7,0.9644,56.0,47.7,...,42.8,23.0,53.2,48.1,40.8,31.4,67.6,11.0,2ND,1.0
3,4,Iowa St.,B12,34,27,113.6,86.5,0.9583,51.9,47.1,...,36.1,35.2,51.7,46.9,34.9,31.5,67.6,6.9,S16,2.0
4,5,Auburn,SEC,34,27,120.7,92.1,0.9573,54.1,43.4,...,38.2,41.0,54.9,42.8,35.2,29.8,69.8,5.5,R64,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,208,Saint Peter's,MAAC,31,19,96.8,100.2,0.4038,45.0,46.6,...,38.1,42.3,42.4,46.4,33.9,31.5,64.0,-8.8,R64,15.0
230,231,Stetson,ASun,31,22,108.3,114.0,0.3559,53.4,52.0,...,31.7,25.9,52.5,52.2,36.5,34.5,66.0,-6.8,R64,16.0
267,268,Howard,MEAC,32,18,104.5,112.9,0.2913,52.0,50.9,...,42.2,39.1,49.2,50.8,37.5,34.0,67.7,-12.1,R68,16.0
271,272,Grambling St.,SWAC,31,20,97.7,106.0,0.2805,48.2,49.1,...,39.6,33.9,47.0,48.4,34.1,33.7,64.9,-8.0,R64,16.0


In [1]:
import torch

In [13]:
cbb23 = pd.read_csv("/Users/jonathan/cmse401/Project/CMSE401-Project/data/cbb23.csv")
cbb23 = cbb23.dropna()
cbb23

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED
0,Kansas,B12,36,28,113.8,92.3,0.9177,52.6,47.0,17.5,...,29.8,31.1,53.0,47.2,34.7,31.0,69.0,10.1,R32,1.0
1,Purdue,B10,35,29,117.9,93.7,0.9333,51.8,47.2,17.2,...,37.2,19.2,54.1,47.2,32.2,31.3,63.9,9.4,R64,1.0
2,Alabama,SEC,37,31,116.1,89.0,0.9548,52.1,41.3,18.7,...,36.6,32.6,53.8,40.8,33.5,28.3,72.6,10.2,S16,1.0
3,Houston,Amer,37,33,117.3,90.3,0.9532,52.3,42.7,15.4,...,28.9,35.7,53.1,43.4,34.0,27.9,63.4,8.0,S16,1.0
4,Texas,B12,38,29,116.1,91.8,0.9371,53.3,48.1,16.3,...,30.5,33.7,54.0,47.9,34.7,32.4,68.5,7.5,E8,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Northern Kentucky,Horz,33,22,102.6,101.4,0.5349,49.1,51.3,17.3,...,26.9,30.5,47.4,49.3,34.3,36.4,61.9,-6.7,R64,16.0
64,Howard,MEAC,32,22,102.7,106.3,0.4031,51.8,50.9,22.8,...,32.2,33.9,49.8,51.1,36.9,33.8,69.0,-7.3,R64,16.0
65,Texas A&M Corpus Chris,Slnd,31,24,105.8,107.4,0.4566,50.5,52.6,17.7,...,34.8,38.2,48.3,54.1,36.4,33.8,68.9,-6.0,R64,16.0
66,Texas Southern,SWAC,33,14,95.4,105.4,0.2394,46.1,49.6,20.3,...,34.3,38.8,47.7,48.9,28.0,33.9,69.2,-13.5,R68,16.0
